In [37]:
from pathlib import Path
import json

# Because notebook is already inside Notebooks/
OUTPUT_DIR = Path("output")

with open(OUTPUT_DIR / "legal_agent_output.json", "r", encoding="utf-8") as f:
    legal_output = json.load(f)["output"]

with open(OUTPUT_DIR / "compliance_agent_output.json", "r", encoding="utf-8") as f:
    compliance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "finance_agent_output.json", "r", encoding="utf-8") as f:
    finance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "operations_agent_output.json", "r", encoding="utf-8") as f:
    operations_output = json.load(f)["output"]

print("✓ Agent outputs loaded successfully")


✓ Agent outputs loaded successfully


## Cross-Verification 
- Grounding Check for Agent Responses
- Ensure all agent responses are grounded in the source documents to prevent hallucinations.
Steps:

1. Define Grounding Verification Functions
2. Cross-Verify Agent Outputs
3. Generate Grounding Report
4. Flag Ungrounded Responses

In [38]:
# Load RAG context chunks from saved RAG search JSON files
RAG_RESULTS_DIR = OUTPUT_DIR / "rag_search_results"

rag_files = sorted(RAG_RESULTS_DIR.glob("*.json"))
print(f"Found {len(rag_files)} RAG search result files")

all_context_chunks = []

for rag_file in rag_files:
    with open(rag_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    for result in data.get("results", []):
        all_context_chunks.append({
            "query": data.get("query", ""),
            "text": result.get("text", ""),
            "contract_id": result.get("contract_id", ""),
            "chunk_index": result.get("chunk_index", -1),
            "score": result.get("score", 0.0),
            "rag_filename": rag_file.name
        })

print(f"✓ Loaded {len(all_context_chunks)} total context chunks")

Found 18 RAG search result files
✓ Loaded 54 total context chunks


In [39]:
from difflib import SequenceMatcher
import re
from typing import Any, Dict, List, Optional, Tuple

def calculate_text_similarity(text1: str, text2: str) -> float:
    if not text1 or not text2:
        return 0.0
    t1 = re.sub(r"\s+", " ", text1.lower().strip())
    t2 = re.sub(r"\s+", " ", text2.lower().strip())
    return SequenceMatcher(None, t1, t2).ratio()


def find_best_match_in_context(claim, context_chunks, threshold=0.4):
    best_score = 0.0
    best_chunk = None

    for chunk in context_chunks:
        chunk_text = chunk["text"]

        # Exact match shortcut
        if claim.lower() in chunk_text.lower():
            return True, 1.0, chunk

        sim = calculate_text_similarity(claim, chunk_text)

        # sentence-level improvement
        for sent in re.split(r"[.!?]+", chunk_text):
            sent = sent.strip()
            if len(sent) > 20:
                sim = max(sim, calculate_text_similarity(claim, sent))

        if sim > best_score:
            best_score = sim
            best_chunk = chunk

    return (best_score >= threshold), best_score, best_chunk


def cross_verify_agent_output(agent_output, context_chunks, agent_name):
    report = {
        "agent": agent_name,
        "total_claims": 0,
        "grounded_claims": 0,
        "ungrounded_claims": 0,
        "details": []
    }

    items = agent_output.get("extracted_clauses", []) + agent_output.get("evidence", [])

    for text in items:
        if not isinstance(text, str) or len(text.strip()) < 10:
            continue

        grounded, score, chunk = find_best_match_in_context(text, context_chunks)

        report["total_claims"] += 1
        if grounded:
            report["grounded_claims"] += 1
        else:
            report["ungrounded_claims"] += 1

        report["details"].append({
            "text": text[:120],
            "grounded": grounded,
            "score": round(score, 3),
            "rag_file": chunk["rag_filename"] if chunk else None,
            "contract_id": chunk["contract_id"] if chunk else None,
            "chunk_index": chunk["chunk_index"] if chunk else None
        })

    if report["total_claims"] > 0:
        report["grounding_score"] = round(report["grounded_claims"] / report["total_claims"], 3)
    else:
        report["grounding_score"] = 0.0

    return report

In [40]:
all_reports = []

print("🔍 Running Cross Verification...\n")

legal_report = cross_verify_agent_output(legal_output, all_context_chunks, "Legal Agent")
compliance_report = cross_verify_agent_output(compliance_output, all_context_chunks, "Compliance Agent")
finance_report = cross_verify_agent_output(finance_output, all_context_chunks, "Finance Agent")
operations_report = cross_verify_agent_output(operations_output, all_context_chunks, "Operations Agent")

all_reports.extend([legal_report, compliance_report, finance_report, operations_report])

for r in all_reports:
    print("="*60)
    print(f"Agent: {r['agent']}")
    print(f"Total Claims: {r['total_claims']}")
    print(f"Grounded: {r['grounded_claims']}")
    print(f"Ungrounded: {r['ungrounded_claims']}")
    print(f"Grounding Score: {r['grounding_score']*100:.1f}%")

🔍 Running Cross Verification...

Agent: Legal Agent
Total Claims: 3
Grounded: 2
Ungrounded: 1
Grounding Score: 66.7%
Agent: Compliance Agent
Total Claims: 2
Grounded: 1
Ungrounded: 1
Grounding Score: 50.0%
Agent: Finance Agent
Total Claims: 4
Grounded: 3
Ungrounded: 1
Grounding Score: 75.0%
Agent: Operations Agent
Total Claims: 4
Grounded: 3
Ungrounded: 1
Grounding Score: 75.0%


In [41]:
from datetime import datetime

verification_report = {
    "timestamp": datetime.now().isoformat(),
    "reports": all_reports
}

output_path = OUTPUT_DIR / "cross_verification_report.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(verification_report, f, indent=2)

print(f"\n✓ Cross-Verification report saved to:")
print(output_path)


✓ Cross-Verification report saved to:
output/cross_verification_report.json


## Coordinator Logic (Rule-Based Routing)

This Coordinator performs:
- Rule-based routing of user queries to relevant agents
- Loads saved agent JSON outputs
- Returns structured outputs without re-running agents

Task completed:
- Added new keyword **"indemnity"** routed to **Finance Agent**
- Tested routing with sample queries

In [42]:

#  — COORDINATOR LOGIC

import json
from pathlib import Path

# STEP 1: Load Existing Agent Outputs 

OUTPUT_DIR = Path("output")   

with open(OUTPUT_DIR / "legal_agent_output.json", "r", encoding="utf-8") as f:
    legal_output = json.load(f)["output"]

with open(OUTPUT_DIR / "compliance_agent_output.json", "r", encoding="utf-8") as f:
    compliance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "finance_agent_output.json", "r", encoding="utf-8") as f:
    finance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "operations_agent_output.json", "r", encoding="utf-8") as f:
    operations_output = json.load(f)["output"]

print("✓ Agent outputs loaded successfully")


#  STEP 2: Define Routing Rules 

ROUTING_RULES = {
    "legal": ["termination", "governing law", "jurisdiction"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "fee", "penalty", "invoice", "indemnity", "indemnification"],  # <-- Added new keyword
    "operations": ["deliverable", "timeline", "sla", "milestone"]
}

print("Routing Rules:")
print(ROUTING_RULES)


#  STEP 3: Define Routing Function 

def route_query(query: str):
    query = query.lower()
    scores = {}

    for agent, keywords in ROUTING_RULES.items():
        scores[agent] = sum(1 for kw in keywords if kw in query)

    # Sort agents by highest match count
    ranked_agents = sorted(scores, key=scores.get, reverse=True)

    # If no keywords matched → default to legal
    if scores[ranked_agents[0]] == 0:
        return ["legal"]

    # Return agents with score > 0
    return [agent for agent in ranked_agents if scores[agent] > 0]


#  STEP 4: Test Routing Logic 

test_queries = [
    "What are the termination clauses?",
    "Explain GDPR audit requirements",
    "What are the payment and invoice terms?",
    "Explain indemnity obligations in the contract",
    "What are the project deliverables and milestones?"
]

print("\nRouting Test Results:")
for q in test_queries:
    print(f"Query: {q}")
    print("Routed Agents:", route_query(q))
    print("-" * 50)


#  STEP 5: Coordinator Execution Logic 

def coordinator_execute(query: str):
    agents = route_query(query)
    results = {}

    for agent in agents:
        if agent == "legal":
            results["legal"] = legal_output

        elif agent == "compliance":
            results["compliance"] = compliance_output

        elif agent == "finance":
            results["finance"] = finance_output

        elif agent == "operations":
            results["operations"] = operations_output

    return results


#  STEP 6: Run Coordinator 

user_query = input("Enter your question about the contract: ")

coordinator_results = coordinator_execute(user_query)

print("\n========== COORDINATOR OUTPUT ==========")
print("Selected Agents:", list(coordinator_results.keys()))

for agent, output in coordinator_results.items():
    print("\n---", agent.upper(), "AGENT OUTPUT ---")
    print(json.dumps(output, indent=2))


✓ Agent outputs loaded successfully
Routing Rules:
{'legal': ['termination', 'governing law', 'jurisdiction'], 'compliance': ['gdpr', 'audit', 'regulatory', 'data protection'], 'finance': ['payment', 'fee', 'penalty', 'invoice', 'indemnity', 'indemnification'], 'operations': ['deliverable', 'timeline', 'sla', 'milestone']}

Routing Test Results:
Query: What are the termination clauses?
Routed Agents: ['legal']
--------------------------------------------------
Query: Explain GDPR audit requirements
Routed Agents: ['compliance']
--------------------------------------------------
Query: What are the payment and invoice terms?
Routed Agents: ['finance']
--------------------------------------------------
Query: Explain indemnity obligations in the contract
Routed Agents: ['finance']
--------------------------------------------------
Query: What are the project deliverables and milestones?
Routed Agents: ['operations']
--------------------------------------------------

========== COORDINAT

## LangGraph Basics

Goals:
- Understand nodes & edges
- Build first LangGraph
- Execute agents through graph

Task Completed:
- Defined shared graph state
- Created Compliance and Legal nodes
- Changed execution order → Compliance → Legal
- Added print logs inside nodes
- Executed graph and observed output order


In [43]:
# LANGGRAPH BASICS

# Step 1: Install & Import LangGraph
import sys
import importlib.util

!"{sys.executable}" -m pip install -q --upgrade --no-deps "langgraph==0.2.76" "langgraph-prebuilt==0.2.3"
!"{sys.executable}" -m pip install -q --upgrade "langgraph-checkpoint" "langgraph-sdk"

print("checkpoint.base present:", importlib.util.find_spec("langgraph.checkpoint.base") is not None)

from langgraph.graph import StateGraph, END
try:
    from langgraph.graph import START
except Exception:
    START = "__start__"

print("✓ LangGraph imported successfully")


# Step 2: Define Shared Graph State
from typing_extensions import TypedDict

class SimpleState(TypedDict, total=False):
    query: str
    compliance: dict
    legal: dict
    trace: list[str]


# Step 3: Define Agent Nodes with Print Logs

def compliance_node(state: SimpleState) -> SimpleState:
    print("\n[COMPLIANCE NODE] --- START ---")
    
    trace = list(state.get("trace", []))
    trace.append("compliance")

    out = {
        "status": "ok",
        "notes": "(demo) Compliance agent reviewed query for GDPR / privacy hints.",
        "seen_query": state.get("query", "")
    }

    print("[COMPLIANCE NODE] Processing Query:", state.get("query"))
    print("[COMPLIANCE NODE] --- END ---")
    
    return {**state, "compliance": out, "trace": trace}


def legal_node(state: SimpleState) -> SimpleState:
    print("\n[LEGAL NODE] --- START ---")
    
    trace = list(state.get("trace", []))
    trace.append("legal")

    out = {
        "status": "ok",
        "notes": "(demo) Legal agent reviewed query for termination / breach / contract terms.",
        "seen_query": state.get("query", "")
    }

    print("[LEGAL NODE] Processing Query:", state.get("query"))
    print("[LEGAL NODE] --- END ---")
    
    return {**state, "legal": out, "trace": trace}


# Step 4 & 5: Build Graph Skeleton + Define Edges
# 📝 Task Requirement: Execution Order = Compliance → Legal

simple_graph = StateGraph(SimpleState)

simple_graph.add_node("compliance_agent", compliance_node)
simple_graph.add_node("legal_agent", legal_node)

# Entry point → Compliance first
simple_graph.set_entry_point("compliance_agent")

# Flow: Compliance → Legal → END
simple_graph.add_edge("compliance_agent", "legal_agent")
simple_graph.add_edge("legal_agent", END)


# Step 6: Compile Graph
simple_app = simple_graph.compile()
print("✓ Graph compiled successfully")


# Step 7: Execute Graph
simple_input: SimpleState = {
    "query": "Review termination clause and GDPR obligations",
    "compliance": {},
    "legal": {},
    "trace": []
}

print("\n Executing LangGraph...\n")
simple_result = simple_app.invoke(simple_input)


# Step 8: Inspect Output
print("\n========== GRAPH OUTPUT ==========")
print("Execution Trace (Order):", simple_result.get("trace"))
print("\nCompliance Output:", simple_result.get("compliance"))
print("\nLegal Output:", simple_result.get("legal"))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


checkpoint.base present: True
✓ LangGraph imported successfully
✓ Graph compiled successfully

 Executing LangGraph...


[COMPLIANCE NODE] --- START ---
[COMPLIANCE NODE] Processing Query: Review termination clause and GDPR obligations
[COMPLIANCE NODE] --- END ---

[LEGAL NODE] --- START ---
[LEGAL NODE] Processing Query: Review termination clause and GDPR obligations
[LEGAL NODE] --- END ---

========== GRAPH OUTPUT ==========
Execution Trace (Order): ['compliance', 'legal']

Compliance Output: {'status': 'ok', 'notes': '(demo) Compliance agent reviewed query for GDPR / privacy hints.', 'seen_query': 'Review termination clause and GDPR obligations'}

Legal Output: {'status': 'ok', 'notes': '(demo) Legal agent reviewed query for termination / breach / contract terms.', 'seen_query': 'Review termination clause and GDPR obligations'}


## Multi-Agent Graph (Nodes & Edges)

**Goals**
- Add all agents as LangGraph nodes  
- Maintain shared state  
- Execute multi-agent pipeline  

**Execution Flow**
legal → compliance → finance → operations → END  

**Student Tasks**
- Change execution order  
- Remove one agent and re-run  
- Observe how shared state changes  


In [44]:
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# 1. Define Expanded Graph State

class MultiAgentState(TypedDict, total=False):
    query: str
    legal: dict
    compliance: dict
    finance: dict
    operations: dict
    trace: list[str]

# Helper to record execution trace

def _append_trace(state: MultiAgentState, label: str) -> list[str]:
    trace = list(state.get("trace", []))
    trace.append(label)
    return trace

# 2. Define Agent Nodes

def legal_agent_node(state: MultiAgentState) -> MultiAgentState:
    print(" [Legal Agent] Start")
    out = {
        "status": "ok",
        "summary": "(demo) Termination / breach clause review",
        "seen_query": state["query"]
    }
    trace = _append_trace(state, "legal")
    print(" [Legal Agent] End")
    return {**state, "legal": out, "trace": trace}


def compliance_agent_node(state: MultiAgentState) -> MultiAgentState:
    print(" [Compliance Agent] Start")
    out = {
        "status": "ok",
        "summary": "(demo) GDPR / privacy compliance review",
        "seen_query": state["query"]
    }
    trace = _append_trace(state, "compliance")
    print(" [Compliance Agent] End")
    return {**state, "compliance": out, "trace": trace}


def finance_agent_node(state: MultiAgentState) -> MultiAgentState:
    print(" [Finance Agent] Start")
    out = {
        "status": "ok",
        "summary": "(demo) Payment / fees / penalties review",
        "seen_query": state["query"]
    }
    trace = _append_trace(state, "finance")
    print(" [Finance Agent] End")
    return {**state, "finance": out, "trace": trace}


def operations_agent_node(state: MultiAgentState) -> MultiAgentState:
    print(" [Operations Agent] Start")
    out = {
        "status": "ok",
        "summary": "(demo) Deliverables / timeline / SLA review",
        "seen_query": state["query"]
    }
    trace = _append_trace(state, "operations")
    print(" [Operations Agent] End")
    return {**state, "operations": out, "trace": trace}

# 3. Register Node Functions

NODE_FUNCS = {
    "legal_agent": legal_agent_node,
    "compliance_agent": compliance_agent_node,
    "finance_agent": finance_agent_node,
    "operations_agent": operations_agent_node,
}

# 4–6. Build Graph with Custom Order

def build_multi_agent_app(order: list[str]):
    graph = StateGraph(MultiAgentState)

    # Add nodes
    for name in order:
        graph.add_node(name, NODE_FUNCS[name])

    # Entry point
    graph.set_entry_point(order[0])

    # Sequential edges
    for current_name, next_name in zip(order, order[1:]):
        graph.add_edge(current_name, next_name)

    graph.add_edge(order[-1], END)

    return graph.compile()

# 7. Execute Multi-Agent Graph (Default Flow)

default_order = ["legal_agent", "compliance_agent", "finance_agent", "operations_agent"]
app_all = build_multi_agent_app(default_order)

input_state: MultiAgentState = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {},
    "trace": []
}

result_all = app_all.invoke(input_state)

print("\n Execution Trace:", result_all["trace"])
print("Result Keys:", list(result_all.keys()))


 [Legal Agent] Start
 [Legal Agent] End
 [Compliance Agent] Start
 [Compliance Agent] End
 [Finance Agent] Start
 [Finance Agent] End
 [Operations Agent] Start
 [Operations Agent] End

 Execution Trace: ['legal', 'compliance', 'finance', 'operations']
Result Keys: ['query', 'legal', 'compliance', 'finance', 'operations', 'trace']


## Student TASK

In [45]:
order_no_operations = ["legal_agent", "compliance_agent", "finance_agent"]
app_no_ops = build_multi_agent_app(order_no_operations)

result_no_ops = app_no_ops.invoke({**input_state, "trace": []})

print("\n Removed Operations Agent")
print("Trace:", result_no_ops["trace"])
print("Operations Output Exists?:", result_no_ops.get("operations"))

 [Legal Agent] Start
 [Legal Agent] End
 [Compliance Agent] Start
 [Compliance Agent] End
 [Finance Agent] Start
 [Finance Agent] End

 Removed Operations Agent
Trace: ['legal', 'compliance', 'finance']
Operations Output Exists?: {}


In [46]:
order_changed = ["compliance_agent", "legal_agent", "finance_agent", "operations_agent"]
app_changed = build_multi_agent_app(order_changed)

result_changed = app_changed.invoke({**input_state, "trace": []})

print("\n🔁 Changed Execution Order")
print("Trace:", result_changed["trace"])


 [Compliance Agent] Start
 [Compliance Agent] End
 [Legal Agent] Start
 [Legal Agent] End
 [Finance Agent] Start
 [Finance Agent] End
 [Operations Agent] Start
 [Operations Agent] End

🔁 Changed Execution Order
Trace: ['compliance', 'legal', 'finance', 'operations']


## Conditional Routing in LangGraph

**Goals**
- Introduce conditional edges  
- Route dynamically based on query  
- Execute only relevant agent instead of running all agents  

**Steps**
1. Define Routing Function  
2. Rebuild Graph with Conditional Entry  
3. Add Conditional Entry Point  
4. Agent → END Edges  
5. Compile Graph  
6. Test Legal Query  
7. Test Finance Query  
8. Test Multiple-Intent Query  

###  Student Task
- Add new keyword mapping  
- Test multiple queries  
- Observe which agent is selected  

In [47]:
from langgraph.graph import StateGraph, END, START

# 1) Define Routing Function


KEYWORD_TO_AGENT = {
    # Legal
    "termination": "legal_agent",
    "breach": "legal_agent",
    "indemn": "legal_agent",       # catches indemnity / indemnification
    "liability": "legal_agent",

    # Compliance
    "gdpr": "compliance_agent",
    "privacy": "compliance_agent",
    "data protection": "compliance_agent",
    "audit": "compliance_agent",

    # Finance
    "invoice": "finance_agent",
    "payment": "finance_agent",
    "late": "finance_agent",
    "penalt": "finance_agent",     # catches penalty / penalties
    "fee": "finance_agent",

    # Operations   (Student-added new mapping)
    "sla": "operations_agent",
    "uptime": "operations_agent",
    "milestone": "operations_agent",
    "deliverable": "operations_agent",
    "timeline": "operations_agent",
}


def route_agent(state: MultiAgentState) -> str:
    query = (state.get("query") or "").lower()

    for keyword, agent_name in KEYWORD_TO_AGENT.items():
        if keyword in query:
            print(f"[ROUTER] matched keyword '{keyword}' → {agent_name}")
            return agent_name

    print("[ROUTER] no keyword match → default legal_agent")
    return "legal_agent"

# 2) Build Conditional Routing Graph

routing_graph = StateGraph(MultiAgentState)

routing_graph.add_node("legal_agent", legal_agent_node)
routing_graph.add_node("compliance_agent", compliance_agent_node)
routing_graph.add_node("finance_agent", finance_agent_node)
routing_graph.add_node("operations_agent", operations_agent_node)

# 3) Conditional Entry Point

routing_graph.add_conditional_edges(
    START,
    route_agent,
    {
        "legal_agent": "legal_agent",
        "compliance_agent": "compliance_agent",
        "finance_agent": "finance_agent",
        "operations_agent": "operations_agent",
    }
)

# 4) Agent → END edges

routing_graph.add_edge("legal_agent", END)
routing_graph.add_edge("compliance_agent", END)
routing_graph.add_edge("finance_agent", END)
routing_graph.add_edge("operations_agent", END)

# 5) Compile Graph

routing_app = routing_graph.compile()

print(" Conditional Routing Graph Compiled")


 Conditional Routing Graph Compiled


In [48]:
def run_routing_test(query: str):
    print("\n" + "="*60)
    print("Query:", query)

    state: MultiAgentState = {
        "query": query,
        "legal": {},
        "compliance": {},
        "finance": {},
        "operations": {},
        "trace": []
    }

    result = routing_app.invoke(state)

    print("Execution Trace:", result.get("trace"))
    print("Result Keys:", list(result.keys()))
    return result

# 6) Test Case 1 — Legal Query

_ = run_routing_test("Review termination clause")


# 7) Test Case 2 — Finance Query

_ = run_routing_test("Check late payment penalties")


# 8) Test Case 3 — Multiple Intent Query (Limitation Demo)

_ = run_routing_test("Check GDPR compliance and payment terms")



Query: Review termination clause
[ROUTER] matched keyword 'termination' → legal_agent
 [Legal Agent] Start
 [Legal Agent] End
Execution Trace: ['legal']
Result Keys: ['query', 'legal', 'compliance', 'finance', 'operations', 'trace']

Query: Check late payment penalties
[ROUTER] matched keyword 'payment' → finance_agent
 [Finance Agent] Start
 [Finance Agent] End
Execution Trace: ['finance']
Result Keys: ['query', 'legal', 'compliance', 'finance', 'operations', 'trace']

Query: Check GDPR compliance and payment terms
[ROUTER] matched keyword 'gdpr' → compliance_agent
 [Compliance Agent] Start
 [Compliance Agent] End
Execution Trace: ['compliance']
Result Keys: ['query', 'legal', 'compliance', 'finance', 'operations', 'trace']


## Conversation Memory & State Persistence (Agent Memory)

**Goals**
- Persist agent outputs in shared graph state  
- Enable agents to read previous results  
- Support multi-step reasoning  

**Steps**
1. Define Graph State with memory field  
2. Initialize memory in input state  
3. Modify agent nodes to write into memory  
4. Build graph with memory support  
5. Compile graph  
6. Execute graph  
7. Verify accumulated memory  

###  Student Task
- Add Legal Agent memory logging  
- Print memory after each node  
- Observe accumulation order  


In [49]:
from typing_extensions import TypedDict, List
from langgraph.graph import StateGraph, END, START

# 1) Define Graph State (with Memory)

class GraphState(TypedDict, total=False):
    query: str
    memory: List[dict]
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


In [50]:
def append_memory(state: GraphState, agent_name: str, output: dict) -> List[dict]:
    memory = list(state.get("memory", []))
    memory.append({
        "agent": agent_name,
        "output": output
    })
    return memory

In [51]:
def legal_agent_node(state: GraphState) -> GraphState:
    print("\n[legal_agent_node] START")

    # Demo output (replace with your real legal agent output if needed)
    out = {
        "status": "ok",
        "summary": "(demo) termination / breach analysis completed",
        "seen_query": state.get("query", "")
    }

    # ---- Add to memory ----
    memory = append_memory(state, "legal_agent", out)

    print("[legal_agent_node] Memory so far:", memory)
    print("[legal_agent_node] END")

    return {**state, "legal": out, "memory": memory}


def compliance_agent_node(state: GraphState) -> GraphState:
    print("\n[compliance_agent_node] START")

    out = {
        "status": "ok",
        "summary": "(demo) GDPR / privacy compliance analysis completed",
        "seen_query": state.get("query", "")
    }

    memory = append_memory(state, "compliance_agent", out)

    print("[compliance_agent_node] Memory so far:", memory)
    print("[compliance_agent_node] END")

    return {**state, "compliance": out, "memory": memory}


def finance_agent_node(state: GraphState) -> GraphState:
    print("\n[finance_agent_node] START")

    out = {
        "status": "ok",
        "summary": "(demo) payment / fee analysis completed",
        "seen_query": state.get("query", "")
    }

    memory = append_memory(state, "finance_agent", out)

    print("[finance_agent_node] Memory so far:", memory)
    print("[finance_agent_node] END")

    return {**state, "finance": out, "memory": memory}


def operations_agent_node(state: GraphState) -> GraphState:
    print("\n[operations_agent_node] START")

    out = {
        "status": "ok",
        "summary": "(demo) SLA / timeline analysis completed",
        "seen_query": state.get("query", "")
    }

    memory = append_memory(state, "operations_agent", out)

    print("[operations_agent_node] Memory so far:", memory)
    print("[operations_agent_node] END")

    return {**state, "operations": out, "memory": memory}

In [52]:
memory_graph = StateGraph(GraphState)

memory_graph.add_node("legal_agent", legal_agent_node)
memory_graph.add_node("compliance_agent", compliance_agent_node)
memory_graph.add_node("finance_agent", finance_agent_node)
memory_graph.add_node("operations_agent", operations_agent_node)

# Sequential Flow
memory_graph.set_entry_point("legal_agent")
memory_graph.add_edge("legal_agent", "compliance_agent")
memory_graph.add_edge("compliance_agent", "finance_agent")
memory_graph.add_edge("finance_agent", "operations_agent")
memory_graph.add_edge("operations_agent", END)

memory_app = memory_graph.compile()

print(" Memory-enabled Multi-Agent Graph Compiled")

 Memory-enabled Multi-Agent Graph Compiled


In [53]:
input_state: GraphState = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "memory": [],          #  Initialize Memory
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = memory_app.invoke(input_state)

print("\n==============================")
print("FINAL MEMORY CONTENT:")
print("==============================")
for step in result["memory"]:
    print(step)


[legal_agent_node] START
[legal_agent_node] Memory so far: [{'agent': 'legal_agent', 'output': {'status': 'ok', 'summary': '(demo) termination / breach analysis completed', 'seen_query': 'Review termination, GDPR compliance, payment terms, and SLAs'}}]
[legal_agent_node] END

[compliance_agent_node] START
[compliance_agent_node] Memory so far: [{'agent': 'legal_agent', 'output': {'status': 'ok', 'summary': '(demo) termination / breach analysis completed', 'seen_query': 'Review termination, GDPR compliance, payment terms, and SLAs'}}, {'agent': 'compliance_agent', 'output': {'status': 'ok', 'summary': '(demo) GDPR / privacy compliance analysis completed', 'seen_query': 'Review termination, GDPR compliance, payment terms, and SLAs'}}]
[compliance_agent_node] END

[finance_agent_node] START
[finance_agent_node] Memory so far: [{'agent': 'legal_agent', 'output': {'status': 'ok', 'summary': '(demo) termination / breach analysis completed', 'seen_query': 'Review termination, GDPR compliance

## Agent-to-Agent Communication & Validation Logic

**Goals**
- Enable agents to read shared memory  
- Refine outputs collaboratively  
- Validate extracted clauses  

**Flow Implemented**
Compliance → Finance → Legal → Operations → END

###  Student Tasks Completed
- Operations Agent reads Legal output  
- Adds validation note for SLA enforceability  
- Memory accumulates cross-agent findings  
- Tested with multi-intent query  


In [54]:
from typing_extensions import TypedDict, List, Optional
from langgraph.graph import StateGraph, END

# 1) Updated Graph State (Shared Knowledge)


class GraphState(TypedDict, total=False):
    query: str
    memory: List[dict]
    validation_notes: List[str]

    legal: Optional[dict]
    compliance: Optional[dict]
    finance: Optional[dict]
    operations: Optional[dict]

# 2) Initialize State


initial_state: GraphState = {
    "query": "Check GDPR compliance, payment penalties, and SLA enforceability",
    "memory": [],
    "validation_notes": [],
    "legal": None,
    "compliance": None,
    "finance": None,
    "operations": None
}

# 3) Compliance Agent (Writes Memory)


def compliance_node(state: GraphState) -> GraphState:
    print("\n[Compliance Agent Running]")

    # Demo output (replace with real compliance_output if available)
    output = {
        "extracted_clauses": ["GDPR data protection obligations clause"],
        "risk_level": "medium"
    }

    state["compliance"] = output

    state["memory"].append({
        "agent": "compliance",
        "findings": output["extracted_clauses"]
    })

    print("Memory after Compliance:", state["memory"])
    return state

# 4) Finance Agent (Reads Compliance Memory)


def finance_node(state: GraphState) -> GraphState:
    print("\n[Finance Agent Running]")

    compliance_findings = [
        m for m in state["memory"] if m["agent"] == "compliance"
    ]

    output = {
        "extracted_clauses": ["Late payment penalty clause"],
        "risk_level": "high"
    }

    state["finance"] = output

    # Validation logic: Finance checks compliance findings
    if compliance_findings:
        state["validation_notes"].append(
            "Finance Agent reviewed Compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": output["extracted_clauses"]
    })

    print("Memory after Finance:", state["memory"])
    return state

# 5) Legal Agent (Final Validation)


def legal_node(state: GraphState) -> GraphState:
    print("\n[Legal Agent Running]")

    output = {
        "extracted_clauses": ["Termination and liability clause"],
        "risk_level": "medium"
    }

    state["legal"] = output

    state["memory"].append({
        "agent": "legal",
        "findings": output["extracted_clauses"]
    })

    print("Memory after Legal:", state["memory"])
    return state

# 6) Operations Agent (Reads Legal Output + SLA Validation)
# Student Task Requirement Implemented Here


def operations_node(state: GraphState) -> GraphState:
    print("\n[Operations Agent Running]")

    legal_findings = [
        m for m in state["memory"] if m["agent"] == "legal"
    ]

    output = {
        "extracted_clauses": ["SLA uptime commitment clause"],
        "risk_level": "low"
    }

    state["operations"] = output

    # 📝 Validation Note for SLA enforceability
    if legal_findings:
        state["validation_notes"].append(
            "Operations Agent validated SLA enforceability against Legal findings."
        )

    state["memory"].append({
        "agent": "operations",
        "findings": output["extracted_clauses"]
    })

    print("Memory after Operations:", state["memory"])
    return state

# 7) Build Collaborative Graph


graph = StateGraph(GraphState)

graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("legal_agent", legal_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")

graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "legal_agent")
graph.add_edge("legal_agent", "operations_agent")
graph.add_edge("operations_agent", END)

# 8) Compile Graph


app = graph.compile()

print(" Collaborative Agent Graph Compiled")

# 9) Execute Collaborative Flow


result = app.invoke(initial_state)

print("\n================ FINAL MEMORY ================")
for item in result["memory"]:
    print(item)

print("\n============= VALIDATION NOTES =============")
for note in result["validation_notes"]:
    print("-", note)


 Collaborative Agent Graph Compiled

[Compliance Agent Running]
Memory after Compliance: [{'agent': 'compliance', 'findings': ['GDPR data protection obligations clause']}]

[Finance Agent Running]
Memory after Finance: [{'agent': 'compliance', 'findings': ['GDPR data protection obligations clause']}, {'agent': 'finance', 'findings': ['Late payment penalty clause']}]

[Legal Agent Running]
Memory after Legal: [{'agent': 'compliance', 'findings': ['GDPR data protection obligations clause']}, {'agent': 'finance', 'findings': ['Late payment penalty clause']}, {'agent': 'legal', 'findings': ['Termination and liability clause']}]

[Operations Agent Running]
Memory after Operations: [{'agent': 'compliance', 'findings': ['GDPR data protection obligations clause']}, {'agent': 'finance', 'findings': ['Late payment penalty clause']}, {'agent': 'legal', 'findings': ['Termination and liability clause']}, {'agent': 'operations', 'findings': ['SLA uptime commitment clause']}]

================ FINAL 

## Compliance Pipeline

Goals:
- Build end-to-end compliance pipeline
- Chain retrieval → analysis → validation
- Produce clean compliance output

In [85]:
from pathlib import Path
import json

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

COMPLIANCE_QUERY = """
Identify clauses related to:
- Regulatory compliance
- Data protection
- Audits and reporting
- Privacy obligations
"""

In [86]:
from sentence_transformers import SentenceTransformer

# Load embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_query(query: str):
    return embed_model.encode(query).tolist()

print("Embedding model loaded")

Embedding model loaded


In [87]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_3CBRV2_8UuBS74QUF6SUv6YJt4h5vHFsn936nc3Sm8JsCQ5eoXZRg3Yerx1ah88PBgH6Nf"

from pinecone import Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])


In [88]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

INDEX_NAME = "cuad-index"   # use your existing index name

# Connect to index
index = pc.Index(INDEX_NAME)

print("Connected to Pinecone index:", INDEX_NAME)


Connected to Pinecone index: cuad-index


In [89]:
def rag_search(query: str, top_k: int = 5):
    # Create embedding
    query_embedding = embed_query(query)

    # Query Pinecone
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    # Collect retrieved chunks
    results = []
    for match in response["matches"]:
        metadata = match["metadata"]
        results.append({
            "score": match["score"],
            "contract_id": metadata.get("contract_id", "unknown"),
            "chunk_id": metadata.get("chunk_id", None),
            "text": metadata.get("text", "")
        })

    return results

In [90]:
test_results = rag_search("test query", top_k=3)

print("Retrieved chunks:", len(test_results))
print(test_results[0]["text"][:200])

Retrieved chunks: 3
15.4. annual test. metavante shall test its plan by conducting one (1) test annually and shall provide customer with a description of the test results in accordance with applicable laws and regulation


In [97]:
from pathlib import Path
import json

# Notebook is inside Notebooks/
OUTPUT_DIR = Path("output")

with open(OUTPUT_DIR / "legal_agent_output.json", "r", encoding="utf-8") as f:
    legal_output = json.load(f)["output"]

with open(OUTPUT_DIR / "compliance_agent_output.json", "r", encoding="utf-8") as f:
    compliance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "finance_agent_output.json", "r", encoding="utf-8") as f:
    finance_output = json.load(f)["output"]

with open(OUTPUT_DIR / "operations_agent_output.json", "r", encoding="utf-8") as f:
    operations_output = json.load(f)["output"]

print("✓ Agent outputs loaded successfully")

✓ Agent outputs loaded successfully


In [ ]:
compliance_response = rag_search(COMPLIANCE_QUERY, top_k=5)

compliance_context = []
for r in compliance_response:          
    compliance_context.append(r["text"])

print("Retrieved Compliance Chunks:", len(compliance_context))


Retrieved Compliance Chunks: 5


In [99]:
combined_compliance_text = "\n\n".join(compliance_context)
print("Combined Compliance Text Length:", len(combined_compliance_text))

Combined Compliance Text Length: 3912


In [100]:
raw_compliance = compliance_output
compliance_pipeline_output = compliance_output

In [101]:
print("Compliance Risk Level:", compliance_pipeline_output["risk_level"])
print("Confidence:", compliance_pipeline_output["confidence"])
print("Extracted Clauses:", len(compliance_pipeline_output["extracted_clauses"]))

Compliance Risk Level: high
Confidence: 1.0
Extracted Clauses: 1


In [102]:
with open(OUTPUT_DIR / "compliance_pipeline_final.json", "w", encoding="utf-8") as f:
    json.dump(compliance_pipeline_output, f, indent=2, ensure_ascii=False)

print("✓ Compliance pipeline final output saved")

✓ Compliance pipeline final output saved
